# Market Data Dependencies in ORE

In [ ]:
import sys
from lxml import etree
import ORE as ore

In [3]:
# Load the XML files and set up the input parameters
ip = ore.InputParameters()
ip.setConventionsFromFile(r"conventions.xml")
ip.setCurveConfigsFromFile(r"curveconfig.xml")
ip.setCurrencyConfigFromFile(r"currencies.xml")
ip.setCalendarAdjustmentFromFile(r"calendar_adjustments.xml")
ip.setPricingEngineFromFile(r"pricingengine.xml")

## Build a Portfolio

Use the new TradeGenerator class in ORE to build a portfolio: see Examples/ORE-Python/TradeGenerator

Full list of supported trade types:

tg.buildSwap("GBP-SONIA", 1000, "1Y", 0.01, True, "SWAP")<br>
tg.buildInflationSwap("UKRPI", 1000, "5Y", "GBP-SONIA", 200, 0.02, True, "inflation swap")<br>
tg.buildFxForward("USD", 1000, "EUR", 1100, "5Y", True)<br>
tg.buildSwap("USD-FedFunds", 1000, "5Y", "USD-LIBOR-3M", 0.01, True)<br>
tg.buildSwap("USD-LIBOR-3M", 1000, "5Y", 0.01, False)<br>
tg.buildCapFloor("USD-LIBOR-3M", 0.01, 1000, "2Y", True, True)<br>
tg.buildEquitySwap("SP5", "Total", 1000, "USD-FedFunds", 1000, "10Y", True)<br>
tg.buildEquitySwap("SP5", "Total", 1000, 0.01, 1000, "6M", True)<br>
tg.buildEquityForward("SP5", 1000, "7Y", 2020)<br>

In [4]:
tg = ore.TradeGenerator(ip.curveConfigs().curveConfigurations()[""], ip.refDataManager(), "CP", "NS")
tg.buildFxForward("USD", 1000, "EUR", 1100, "5Y", True)
tg.buildSwap("USD-FedFunds", 1000, "5Y", "USD-LIBOR-3M", 0.01, True)
tg.buildEquityForward("RIC:.STOXX50", 1000, "1Y", 2020)
ip.setPortfolio(tg)

print(etree.tostring(etree.fromstring(tg.toXMLString()), pretty_print=True).decode())

<Portfolio>
	<Trade id="1_USD-EUR_FX_FORWARD">
		<TradeType>FxForward</TradeType>
		<Envelope>
			<CounterParty>CP</CounterParty>
			<NettingSetId>NS</NettingSetId>
			<PortfolioIds/>
			<AdditionalFields/>
		</Envelope>
		<FxForwardData>
			<ValueDate>2030-06-06</ValueDate>
			<BoughtCurrency>EUR</BoughtCurrency>
			<BoughtAmount>1100</BoughtAmount>
			<SoldCurrency>USD</SoldCurrency>
			<SoldAmount>1000</SoldAmount>
			<Settlement>Physical</Settlement>
			<SettlementData>
				<Rules/>
			</SettlementData>
		</FxForwardData>
	</Trade>
	<Trade id="2_USD-FedFunds_USD-LIBOR-3M_SWAP">
		<TradeType>Swap</TradeType>
		<Envelope>
			<CounterParty>CP</CounterParty>
			<NettingSetId>NS</NettingSetId>
			<PortfolioIds/>
			<AdditionalFields/>
		</Envelope>
		<SwapData>
			<LegData>
				<LegType>Floating</LegType>
				<Payer>false</Payer>
				<Currency>USD</Currency>
				<PaymentConvention>F</PaymentConvention>
				<DayCounter>Actual/360</DayCounter>
				<Notionals>
					<Notional>1000</Notional

## Portfolio Analyser

Call PortfolioAnalyser and show the curve dependencies gererated from it

In [5]:
pa = ore.PortfolioAnalyser(ip.portfolio(), ip.pricingEngine(), "USD", ip.curveConfigs().curveConfigurations()[""])
marketObjects = pa.allMarketObjects()

# Loop over allMarketObjects and print the keys and values
for key, value in marketObjects.items():
    for k, v in value.items():
        l = ore.MarketObjectString(ore.derefMarketObject(k))
        for s in v:
            print(f"Config: {key}, Type: {l}, Curve: {s}")

Config: default, Type: DiscountCurve, Curve: EUR
Config: default, Type: DiscountCurve, Curve: USD
Config: default, Type: IndexCurve, Curve: EUR-EONIA
Config: default, Type: IndexCurve, Curve: EUR-ESTER
Config: default, Type: IndexCurve, Curve: USD-FedFunds
Config: default, Type: IndexCurve, Curve: USD-LIBOR-3M
Config: default, Type: IndexCurve, Curve: USD-SOFR
Config: default, Type: FXSpot, Curve: EURUSD
Config: default, Type: FXSpot, Curve: USDEUR
Config: default, Type: EquityCurves, Curve: RIC:.STOXX50


## Portfolio Details Analytic

In [6]:
# Run portfolio details analytic

app = ore.OREApp(ip, "", 63, False, False)
app.setupLog(63, "Output", "log_pd.txt")
ip.insertAnalytic("PORTFOLIO_DETAILS")
ip.setBaseCurrency("USD")
md = ore.MarketDataInMemoryLoader(ip, ore.StrVector(), ore.StrVector())
app.run(md)

In [7]:
app.getReportNames()

('counterparties',
 'market_objects',
 'netting_sets',
 'pricingstats',
 'risk_factors',
 'runtimes',
 'swap_indices',
 'todaysmarketcalibration',
 'trade_types',
 'underlying_indices')

In [8]:
import utilities
mo = utilities.report_to_df(app.getReport('market_objects'))
mo

,MarketObjectType,MarketObjectName
0,DiscountCurve,EUR|USD
1,IndexCurve,EUR-EONIA|EUR-ESTER|USD-FedFunds|USD-LIBOR-3M|...
2,FXSpot,EURUSD|USDEUR
3,EquityCurves,RIC:.STOXX50


In [9]:
rf = utilities.report_to_df(app.getReport('risk_factors'))
rf

,RiskFactorType,RiskFactorName
0,DiscountCurve,EUR|USD
1,IndexCurve,USD-FedFunds|USD-LIBOR-3M|USD-SOFR
2,FXSpot,EURUSD
3,EquitySpot,RIC:.STOXX50
4,DividendYield,RIC:.STOXX50


## Build Todays Market Parameters

In [10]:
# Build a todaysMarketParameters object
tmp = ore.TodaysMarketParameters()

# set up the configurations
for c in marketObjects.keys():
    tmp.addConfiguration(c, ore.MarketConfiguration());
base_ccy = "USD"
for config, m in marketObjects.items():
    for k, v in m.items():
        mkt_obj = ore.derefMarketObject(k)
        mo_map = {}
        for s in v:
            curve = s
            if ore.MarketObjectString(mkt_obj) == "DiscountCurve":
                curve = ore.currencyToDiscountCurve(curve, base_ccy, "", ip.curveConfigs().curveConfigurations()[""]);
            curve_spec = ore.marketObjectToCurveSpec(mkt_obj, curve, base_ccy, ip.curveConfigs().curveConfigurations()[""])
            mo_map[s] = curve_spec
        tmp.addMarketObject(mkt_obj, config, mo_map)

print(etree.tostring(etree.fromstring(tmp.toXMLString()), pretty_print=True).decode())

<TodaysMarket>
	<Configuration id="default">
		<YieldCurvesId>default</YieldCurvesId>
		<DiscountingCurvesId>default</DiscountingCurvesId>
		<IndexForwardingCurvesId>default</IndexForwardingCurvesId>
		<SwapIndexCurvesId>default</SwapIndexCurvesId>
		<ZeroInflationIndexCurvesId>default</ZeroInflationIndexCurvesId>
		<YYInflationIndexCurvesId>default</YYInflationIndexCurvesId>
		<FxSpotsId>default</FxSpotsId>
		<FxVolatilitiesId>default</FxVolatilitiesId>
		<SwaptionVolatilitiesId>default</SwaptionVolatilitiesId>
		<YieldVolatilitiesId>default</YieldVolatilitiesId>
		<CapFloorVolatilitiesId>default</CapFloorVolatilitiesId>
		<CDSVolatilitiesId>default</CDSVolatilitiesId>
		<DefaultCurvesId>default</DefaultCurvesId>
		<YYInflationCapFloorVolatilitiesId>default</YYInflationCapFloorVolatilitiesId>
		<ZeroInflationCapFloorVolatilitiesId>default</ZeroInflationCapFloorVolatilitiesId>
		<EquityCurvesId>default</EquityCurvesId>
		<EquityVolatilitiesId>default</EquityVolatilitiesId>
		<Securitie

## Identify Required Market Data and Fixings

In [11]:
mdl = ore.DummyMarketDataLoader(ip)
dts = [ore.parseDate("2025-05-28")]
mdl.populateLoader(tmp, dts)
quotes = mdl.marketDataQuotes()
fixings = mdl.marketFixings()

In [12]:
for q in quotes:
    print(f"{q[0]}, {q[1]}")

2025-05-28, BASIS_SWAP/BASIS_SPREAD/1D/1D/USD/SOFR_FedFunds/10Y
2025-05-28, BASIS_SWAP/BASIS_SPREAD/1D/1D/USD/SOFR_FedFunds/11Y
2025-05-28, BASIS_SWAP/BASIS_SPREAD/1D/1D/USD/SOFR_FedFunds/12Y
2025-05-28, BASIS_SWAP/BASIS_SPREAD/1D/1D/USD/SOFR_FedFunds/15M
2025-05-28, BASIS_SWAP/BASIS_SPREAD/1D/1D/USD/SOFR_FedFunds/15Y
2025-05-28, BASIS_SWAP/BASIS_SPREAD/1D/1D/USD/SOFR_FedFunds/18M
2025-05-28, BASIS_SWAP/BASIS_SPREAD/1D/1D/USD/SOFR_FedFunds/1Y
2025-05-28, BASIS_SWAP/BASIS_SPREAD/1D/1D/USD/SOFR_FedFunds/20Y
2025-05-28, BASIS_SWAP/BASIS_SPREAD/1D/1D/USD/SOFR_FedFunds/21M
2025-05-28, BASIS_SWAP/BASIS_SPREAD/1D/1D/USD/SOFR_FedFunds/25Y
2025-05-28, BASIS_SWAP/BASIS_SPREAD/1D/1D/USD/SOFR_FedFunds/2Y
2025-05-28, BASIS_SWAP/BASIS_SPREAD/1D/1D/USD/SOFR_FedFunds/30Y
2025-05-28, BASIS_SWAP/BASIS_SPREAD/1D/1D/USD/SOFR_FedFunds/3Y
2025-05-28, BASIS_SWAP/BASIS_SPREAD/1D/1D/USD/SOFR_FedFunds/40Y
2025-05-28, BASIS_SWAP/BASIS_SPREAD/1D/1D/USD/SOFR_FedFunds/4Y
2025-05-28, BASIS_SWAP/BASIS_SPREAD/1D/1D/US

In [13]:
for f in fixings:
    print(f"{f[0][0]}, {f[0][1]}, {f[1]}")

2025-01-28, EUR-EONIA, False
2025-01-29, EUR-EONIA, False
2025-01-30, EUR-EONIA, False
2025-01-31, EUR-EONIA, False
2025-02-03, EUR-EONIA, False
2025-02-04, EUR-EONIA, False
2025-02-05, EUR-EONIA, False
2025-02-06, EUR-EONIA, False
2025-02-07, EUR-EONIA, False
2025-02-10, EUR-EONIA, False
2025-02-11, EUR-EONIA, False
2025-02-12, EUR-EONIA, False
2025-02-13, EUR-EONIA, False
2025-02-14, EUR-EONIA, False
2025-02-17, EUR-EONIA, False
2025-02-18, EUR-EONIA, False
2025-02-19, EUR-EONIA, False
2025-02-20, EUR-EONIA, False
2025-02-21, EUR-EONIA, False
2025-02-24, EUR-EONIA, False
2025-02-25, EUR-EONIA, False
2025-02-26, EUR-EONIA, False
2025-02-27, EUR-EONIA, False
2025-02-28, EUR-EONIA, False
2025-03-03, EUR-EONIA, False
2025-03-04, EUR-EONIA, False
2025-03-05, EUR-EONIA, False
2025-03-06, EUR-EONIA, False
2025-03-07, EUR-EONIA, False
2025-03-10, EUR-EONIA, False
2025-03-11, EUR-EONIA, False
2025-03-12, EUR-EONIA, False
2025-03-13, EUR-EONIA, False
2025-03-14, EUR-EONIA, False
2025-03-17, EU